# Flowsheet optimization
## ROK model: M5; Region: Eagle Ford Basin (EF-Basin); CO$_2$ tax rate: USD 45/tonne CO$_2$e

In [1]:
# simulation case specifications
model_code = 5
case_name = 'EF-Basin'
c_tax_rate = 4.5e-2

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                    define_models, \
                                    define_arcs, \
                                    set_unit_model_variables, \
                                    initialize_flowsheet, \
                                    set_scaling_factors,\
                                    update_model_after_initialization,\
                                    vapor_only_to_vapor_liquid_reformulate,\
                                    update_model_for_optimization, \
                                    unfix_DOFs_pre_optimization, \
                                    fix_DOFs_post_optimization

## Create flowsheet

In [3]:
m = create_flowsheet(model_code)

In [4]:
M_catalyst = 1167.003367 # kg

## Define equipment and connections

In [5]:
# define unit models
define_models(m, catalyst_mass = M_catalyst)
#define connections
define_arcs(m)

## Define inlet compositions

In [6]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_composition_dict = {}

for col in inlet_df.columns:
    if col == case_name:
        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)

inlet_flow_rate = 481.3888889

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188}

## Define constraints and set-points for equipment

In [7]:
set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)

## Scale model components

In [8]:
if model_code == 2 or model_code == 3:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
elif model_code == 4 or model_code == 5:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
else:
    pass

## Read-in initialization data from .json file with Bakken feed

In [9]:
init_file_name = "./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format('Bakken', model_code)
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.07553577423095703,
 'etime_read_dict': 0.17581629753112793,
 'etime_read_suffixes': 0.0049915313720703125}

## Add post-initialization constraints

In [10]:
update_model_after_initialization(m)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)

Liquid phase flow is zero. Translator block fs.T102 liquid phase composition equations are being modified...

Degenerate constraints removed and liquid phase composition values set to 1e-8

Translator block fs.T102 liquid phase composition equations already modified, no degenerate constraints remaining.



## Read-in flowsheet convergence data from .json file

In [11]:
init_file_name = "./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.08616781234741211,
 'etime_read_dict': 0.17053794860839844,
 'etime_read_suffixes': 0.005975008010864258}

## Costing

In [12]:
from src.emissions_calculations import calc_lhv_values, calculate_stream_energies, calculate_emissions, create_ghg_objective, delete_region_specific_components
from src.costing_function import add_costing,calculate_costs_for_objective
from src.utility_minimization_1d import (
    min_utility,
    PinchDataClass,
    heat_ex_data,
    gen_curves,
    print_HX_results,
    generate_curves,
    heat_data,
    pinch_calc,
    return_data
)

### Equipment costing

In [13]:
add_costing(m)

### Heat integration

In [14]:
### Heat Exchangers and reactors
min_utility(
    m.fs, [m.fs.H101, m.fs.H103, m.fs.R101], [m.fs.H102, m.fs.H104, m.fs.H105, m.fs.H106, m.fs.R102], 10.0
)
m.fs.Qs.fix()

### Emissions calculation

In [15]:
calc_lhv_values(m,case_name,'./LHV.xlsx','NGL_compositions.csv','NGL_fraction.csv')
calculate_stream_energies(m)
calculate_emissions(m,case_name,'emissions_factor_by_region.csv')
create_ghg_objective(m)
calculate_costs_for_objective(m,c_tax_flag=True, c_tax_val = c_tax_rate)

## Read-in costing-initialized flowsheet data from .json file

In [16]:
init_file_name = "./initialization_files/CISTAR_solve_with_costing_{}_C_tax_{}_M{}_purge_{}.json.gz".format(case_name,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.11247038841247559,
 'etime_read_dict': 0.3111395835876465,
 'etime_read_suffixes': 0.010613441467285156}

## Flowsheet optimization with 8 DOFs

In [17]:
update_model_for_optimization(m)

Model updates:


Constraint added for recycle ratio <= 8.0.

fs.H103 temperature lower and upper bounds added.

fs.R102 isothermal operation constraint added.

fs.H106 temperature lower bound updated.

fs.F102 temperature lower bound updated.

Objective function added.



## Dataframes to record multiple zone output

In [18]:
import pandas as pd

In [19]:
cols = ['region','butene', 'pentene', 'hexene', 'heptene', 'octene', 'nonene']
outlet_conc_df = pd.DataFrame(columns=cols)

LHV_cols = ['region','LHV_kJ_mol','MW_fuel','LHV_kJ_s']
LHV_df = pd.DataFrame(columns=LHV_cols)

In [20]:
## emissions factors for upstream emissions calculations
upstream_emissions_factor_df = pd.read_csv('emissions_factor_by_region.csv')

In [21]:
for col in inlet_df.columns:
    if col == 'Species':
        pass
    elif col == 'EF-Basin': # only EF-Basin composition solved here since this solution is used to initialize optimization of EF-1 to EF-12
        print('\n Region = {} \n'.format(col))
        region = col
        case_name_current = col
        
        print("Upstream EF = ",m.fs.upstream_emission_factor())
        
        # Unfix DOFs
        unfix_DOFs_pre_optimization(m)
        
        m.fs.H103.outlet.temperature.fix(593.0)
        m.fs.H103.outlet.temperature.unfix()
        DOF_initial = degrees_of_freedom(m)
        print("The initial DOF is {0}".format(DOF_initial))
        
        # Solve optimization problem
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6,
                          'bound_push': 1e-8,
                          'max_iter': 500,
                         }
        solve_status = solver.solve(m, tee=True)

        # Fix DOFs
        fix_DOFs_post_optimization(m)
        
        ms.to_json(m, fname="./initialization_files/CISTAR_optimal_solution_{}_C_tax_{}_M{}_purge_{}_sequential_solve.json.gz".format(case_name_current,m.fs.c_tax_rate(), model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))
        
        conc_list = []
        for col in cols:
            if col == 'region':
                conc_list.append(region)
            else:
                conc_list.append(m.fs.F102.liq_outlet.flow_mol_phase_comp[0.0, 'Liq', col]())

        outlet_conc_df.loc[len(outlet_conc_df.index)] = conc_list
        
        print("====================  Region = {}  ======================".format(region))
        print(outlet_conc_df)
    else:
        pass


 Region = EF-Basin 

Upstream EF =  9.1
The initial DOF is 8
Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
